<img src='https://www.unifor.br/o/unifor-theme/images/unifor-logo-horizontal.svg' width="250px">

# DATA HARVESTING / Projeto da Disciplina

Prof.: Ms. Alex Lima<br>
MBA em Ciência de Dados<br>
Universidade de Fortaleza

**Gabriela Ferreira Coutinho - 2418581**</br>

# 1. Capturando as URLs para Web Scraping

## 1.1 Instalação e Configuração 

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# 🔹 Configuração do Selenium para evitar bloqueios
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

# 🔹 Inicializa o WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

print("✅ Selenium configurado com sucesso!")



✅ Selenium configurado com sucesso!


## 1.2 Acessar a Página Principal

In [16]:
# 🔹 URL da página principal do torneio
url_base = "https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7"
driver.get(url_base)

# 🔹 Espera a página carregar totalmente
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(5)  # Tempo extra para garantir carregamento

print("✅ Página carregada com sucesso!")

✅ Página carregada com sucesso!


## 1.3 Abrir o Dropdown e Coletar as Temporadas

In [17]:
# 🔹 Clicar no botão do dropdown para abrir a lista de temporadas
try:
    dropdown_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "DropdownButton"))
    )
    dropdown_button.click()
    time.sleep(3)  # Espera os itens carregarem
except Exception as e:
    print("🚨 Erro ao abrir o dropdown das temporadas:", e)
    driver.quit()
    exit()

# 🔹 Coletar os IDs das temporadas
temporadas_urls = {}

try:
    # 🔹 Buscar a lista de temporadas toda vez antes de interagir
    temporadas_elements = driver.find_elements(By.XPATH, "//ul[@role='listbox']/li")
    temporadas_textos = [item.text.strip() for item in temporadas_elements]

    for nome_temporada in temporadas_textos:
        # 🔹 Reabre o dropdown para garantir que ele ainda está disponível
        dropdown_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "DropdownButton"))
        )
        dropdown_button.click()
        time.sleep(2)

        # 🔹 Rebusca a lista de temporadas antes de clicar
        temporadas_elements = driver.find_elements(By.XPATH, "//ul[@role='listbox']/li")

        for item in temporadas_elements:
            if item.text.strip() == nome_temporada:
                item.click()  # Seleciona a temporada para carregar a página correta
                time.sleep(3)

                # 🔹 Coletar o ID da URL após a seleção
                temporada_id = driver.current_url.split("#id:")[-1]
                url_final = f"https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:{temporada_id}"
                temporadas_urls[nome_temporada] = url_final
                break

except Exception as e:
    print("🚨 Erro ao coletar as temporadas:", e)

print("✅ Temporadas coletadas com sucesso!")


✅ Temporadas coletadas com sucesso!


## 1.4 Mostrar as 10 Temporadas Coletadas

In [18]:
# 🔹 Filtrar apenas as 10 últimas temporadas
temporadas_filtradas = dict(list(temporadas_urls.items())[:10])

# 🔹 Exibe os IDs e URLs corrigidas
print("\n📌 Temporadas encontradas no Sofascore:")
for temporada, url in temporadas_filtradas.items():
    print(f"{temporada}: {url}")



📌 Temporadas encontradas no Sofascore:
23/24: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:52162
22/23: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:41897
21/22: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:36886
20/21: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:29267
19/20: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:23766
18/19: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:17351
17/18: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:13415
16/17: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:11773
15/16: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:10390
14/15: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:8226


## 1.5 Fechar o navegador 

In [19]:
# 🔹 Fechar o navegador ao fim do processo
driver.quit()
print("✅ Navegador fechado com sucesso!")

✅ Navegador fechado com sucesso!


## 1.6 Salvar temporadas como CSV

In [20]:
import os
import pandas as pd

# 🔹 Criar a pasta 'data' caso não exista
os.makedirs("data", exist_ok=True)

# 🔹 Caminho do arquivo CSV
csv_path = "data/temporadas_urls.csv"

# 🔹 Converter dicionário para DataFrame e salvar
df_temporadas = pd.DataFrame(list(temporadas_filtradas.items()), columns=["Temporada", "URL"])
df_temporadas.to_csv(csv_path, index=False, encoding="utf-8")

print(f"✅ Temporadas salvas em {csv_path}!")


✅ Temporadas salvas em data/temporadas_urls.csv!


# 2. Capturar dados dos jogadores das temporadas

## 2.1 Carregar as Temporadas do CSV

In [35]:
import pandas as pd

# 🔹 Caminho do arquivo CSV salvo anteriormente
csv_path = "data/temporadas_urls.csv"

# 🔹 Carregar o CSV e transformar em dicionário {Temporada: URL}
df_temporadas = pd.read_csv(csv_path)
temporadas_urls = dict(zip(df_temporadas["Temporada"], df_temporadas["URL"]))

print(f"✅ {len(temporadas_urls)} temporadas carregadas do CSV para scraping.")


✅ 10 temporadas carregadas do CSV para scraping.


## 2.2 Função para Coletar Dados de uma Temporada

### 2.2.1 Configuração Inicial

In [36]:
# 🔹 Importação das bibliotecas necessárias
import os
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 🔹 Configuração do Selenium para evitar bloqueios
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")

# 🔹 Inicializa o WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🔹 Criar a pasta "data" para armazenar os resultados
os.makedirs("data", exist_ok=True)

print("✅ Configuração inicial concluída!")


✅ Configuração inicial concluída!


### 2.2.2 Carregar Temporadas do CSV

In [37]:
# 🔹 Carregar as URLs das temporadas do arquivo CSV
csv_path = "data/temporadas_urls.csv"

if os.path.exists(csv_path):
    df_temporadas = pd.read_csv(csv_path)
    temporadas_urls = dict(zip(df_temporadas["Temporada"], df_temporadas["URL"]))
    print(f"✅ {len(temporadas_urls)} temporadas carregadas do CSV para scraping.")
else:
    print("❌ Arquivo de temporadas não encontrado. Execute a etapa de coleta de URLs primeiro!")


✅ 10 temporadas carregadas do CSV para scraping.


### 2.2.3 Função para Coletar Estatísticas dos Jogadores

In [38]:
def coletar_dados_temporada(temporada, url):
    print(f"\n🔄 Acessando temporada {temporada}: {url}")

    # Inicializa as variáveis antes do scraping
    dados_totais = []
    cabecalhos = []

    # 🟢 **1º Passo: Coletar a primeira página com Requests + BeautifulSoup**
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        tabela = soup.find("table")

        if tabela:
            print("✅ Tabela encontrada com BeautifulSoup")

            # Captura os cabeçalhos da tabela
            cabecalhos = [th.text.strip() for th in tabela.find_all("th")]
            cabecalhos.insert(1, "Time")  # Adiciona a coluna "Time"

            # Captura os dados da primeira página
            for linha in tabela.find_all("tr")[1:]:
                colunas = linha.find_all("td")
                if colunas:
                    try:
                        time_element = colunas[1].find("img")
                        nome_time = time_element["alt"] if time_element else "Desconhecido"
                    except:
                        nome_time = "Desconhecido"

                    dados_linha = [coluna.text.strip() for coluna in colunas]
                    dados_linha.insert(1, nome_time)
                    dados_totais.append(dados_linha)

            print("✅ Dados da primeira página coletados com Requests + BeautifulSoup")
        else:
            print("⚠️ Tabela não encontrada com Requests, tentando Selenium...")

    else:
        print(f"⚠️ Erro ao carregar a página ({response.status_code}), tentando Selenium...")

    # 🟡 **2º Passo: Coletar as próximas páginas com Selenium**
    driver.get(url)
    time.sleep(5)

    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )

        while True:
            try:
                tabela = driver.find_element(By.TAG_NAME, "table")

                # Capturar cabeçalhos apenas se não foram definidos antes
                if not cabecalhos:
                    cabecalhos = [th.text for th in tabela.find_elements(By.TAG_NAME, "th")]
                    cabecalhos.insert(1, "Time")

                # Capturar os dados das páginas
                linhas = tabela.find_elements(By.TAG_NAME, "tr")
                for linha in linhas[1:]:
                    colunas = linha.find_elements(By.TAG_NAME, "td")
                    if colunas:
                        try:
                            time_element = colunas[1].find_element(By.TAG_NAME, "img")
                            nome_time = time_element.get_attribute("alt") if time_element else "Desconhecido"
                        except:
                            nome_time = "Desconhecido"

                        dados_linha = [coluna.text for coluna in colunas]
                        dados_linha.insert(1, nome_time)
                        dados_totais.append(dados_linha)

                # Verificar o botão de próxima página
                try:
                    botao_proximo = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@style, 'justify-content: flex-end')]"))
                    )
                    if "disabled" in botao_proximo.get_attribute("class"):
                        break

                    botao_proximo.click()
                    time.sleep(random.uniform(2, 4))

                except:
                    print("📌 Última página alcançada.")
                    break

            except Exception as e:
                print(f"⚠️ Erro ao capturar dados da temporada {temporada}: {e}")
                break

        # **Salvar os dados em CSV**
        if dados_totais:
            df = pd.DataFrame(dados_totais, columns=cabecalhos)
            nome_arquivo = f"data/estatisticas_jogadores_{temporada.replace('/', '-')}.csv"
            df.to_csv(nome_arquivo, index=False, encoding="utf-8")
            print(f"✅ Dados da temporada {temporada} salvos em {nome_arquivo}")
        else:
            print(f"❌ Nenhum dado foi coletado para a temporada {temporada}.")

    except Exception as e:
        print(f"🚨 Erro ao carregar a página da temporada {temporada}: {e}")

### 2.2.4 Executar Web Scraping para Todas as Temporadas

In [39]:
# 🔹 Executar a coleta de estatísticas para todas as temporadas
for temporada, url in temporadas_urls.items():
    coletar_dados_temporada(temporada, url)

# 🔹 Fechar o navegador ao final do processo
driver.quit()
print("✅ Web scraping concluído! Navegador fechado.")



🔄 Acessando temporada 23/24: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:52162
⚠️ Tabela não encontrada com Requests, tentando Selenium...
📌 Última página alcançada.
✅ Dados da temporada 23/24 salvos em data/estatisticas_jogadores_23-24.csv

🔄 Acessando temporada 22/23: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:41897
⚠️ Tabela não encontrada com Requests, tentando Selenium...
📌 Última página alcançada.
✅ Dados da temporada 22/23 salvos em data/estatisticas_jogadores_22-23.csv

🔄 Acessando temporada 21/22: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:36886
⚠️ Tabela não encontrada com Requests, tentando Selenium...
📌 Última página alcançada.
✅ Dados da temporada 21/22 salvos em data/estatisticas_jogadores_21-22.csv

🔄 Acessando temporada 20/21: https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:29267
⚠️ Tabela não encontrada com Requests, tentando Seleni